In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import r2_score
import random
from scipy.signal import savgol_filter
from sklearn.metrics import mean_squared_error
import os

In [ ]:
# Function to estimate log-normal distribution parameters from mean and median
def estimate_lognormal_params(mean, median):
    sigma = np.sqrt(2 * np.log(mean / median))
    mu = np.log(median)
    return mu, sigma

# Function to generate synthetic data using log-normal distribution and clip to observed range
def generate_lognormal_samples(mean, median, min_val, max_val, size=10000):
    mu, sigma = estimate_lognormal_params(mean, median)
    samples = np.random.lognormal(mean=mu, sigma=sigma, size=size)
    return np.clip(samples, min_val, max_val)


def generate_time (t_initial, t_final, n_timesteps):
    t_numpy = np.linspace(t_initial, t_final, n_timesteps)
    t_reshape = t_numpy.reshape(1, -1)
    t_torch = torch.tensor(t_reshape, dtype=torch.float32).to(device)
    return t_numpy, t_reshape, t_torch

def zscore(x, eps=1e-12):
    x = np.asarray(x, dtype=float)
    mu = x.mean()
    sd = x.std()
    if sd < eps:
        return x * 0.0
    return (x - mu) / sd

def best_lag_correlation(a, b, max_lag=None, normalize='zscore'):

    a = np.asarray(a, dtype=float)
    b = np.asarray(b, dtype=float)

    T = min(len(a), len(b))
    a = a[:T]
    b = b[:T]

    if normalize == 'zscore':
        a_n = zscore(a)
        b_n = zscore(b)
    else:
        a_n = a
        b_n = b

    if max_lag is None:
        max_lag = T - 1

    best_corr = -np.inf
    best_lag = 0
    best_pair = (None, None)


    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:

            a_slice = a_n[-lag:]
            b_slice = b_n[:T+lag]
        elif lag > 0:

            a_slice = a_n[:T-lag]
            b_slice = b_n[lag:]
        else:
            a_slice = a_n
            b_slice = b_n

        if len(a_slice) < 2:
            continue

        corr = np.dot(a_slice, b_slice) / (len(a_slice) - 1)

        if corr > best_corr:
            best_corr = corr
            best_lag = lag

            if lag < 0:
                a_raw = a[-lag:]
                b_raw = b[:T+lag]
            elif lag > 0:
                a_raw = a[:T-lag]
                b_raw = b[lag:]
            else:
                a_raw = a
                b_raw = b
            best_pair = (a_raw, b_raw)

    return {'lag': best_lag, 'corr': best_corr, 'a_seg': best_pair[0], 'b_seg': best_pair[1]}

def smooth_peak(curve, window_length=11, polyorder=3, baseline_ratio=0.01):
    curve = np.asarray(curve)
    smoothed = savgol_filter(curve, window_length=window_length, polyorder=polyorder)

    peak_idx = np.argmax(smoothed)
    peak_val = smoothed[peak_idx]
    threshold = peak_val * baseline_ratio

    # Find start (left of peak)
    start_idx = 0
    for i in range(peak_idx, 0, -1):
        if smoothed[i] < threshold:
            start_idx = i
            break

    # Find end (right of peak)
    end_idx = len(curve) - 1
    for i in range(peak_idx, len(curve)):
        if smoothed[i] < threshold:
            end_idx = i
            break

    # Keep only the main peak region
    cleaned = np.zeros_like(curve)
    cleaned[start_idx:end_idx + 1] = curve[start_idx:end_idx + 1]

    return cleaned

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Generate synthetic values for each variable
mass     = generate_lognormal_samples(mean=9.62,     median=2.81,     min_val=0.0404,    max_val=108.86)
area     = generate_lognormal_samples(mean=224.82,   median=17.09,    min_val=0.98,      max_val=4931.81)
velocity = generate_lognormal_samples(mean=0.51,     median=0.40,     min_val=0.04,      max_val=1.58)
distance = generate_lognormal_samples(mean=46162.12, median=29611.86, min_val=35.41,     max_val=294509.22)
dispersion = generate_lognormal_samples(mean=130.73,     median=37.48,     min_val=1.9,      max_val=1486.45)

In [ ]:
# Linear scale and reshape
M = mass.reshape(-1,1)
A = area.reshape(-1,1)
x = distance.reshape(-1,1)
u = velocity.reshape(-1,1)
Q = np.array(A*u)
Q = Q.reshape(-1,1)
R= 1
theta = 1
D = dispersion.reshape(-1,1)

In [ ]:
n_timesteps = 100000
t_numpy, t_reshape, t_torch = generate_time(1, 3e6, n_timesteps)
c_downstream_filter = (1e6 * M) / (2 * theta * A * R * np.sqrt(np.pi * D * t_reshape/ R)) * np.exp(-((x - u * t_reshape / R) ** 2) / (4 * D * t_reshape / R))

In [ ]:
t_peak = []
max_concentrion_array = []
for i in range (c_downstream_filter.shape[0]):
  max_concentrion_idx = np.argmax(c_downstream_filter[i,:])
  max_concentrion = np.max(c_downstream_filter[i,:])
  time_peak = t_reshape[:,max_concentrion_idx]
  t_peak = np.append(t_peak,time_peak)
  max_concentrion_array = np.append(max_concentrion_array,max_concentrion)

v = distance/t_peak
pe = (distance*velocity)/dispersion
valid_v = (v >= 0.1) & (v <= 6)
valid_pe = (pe >= 10) & (pe <= 1500)
valid_max_c = (max_concentrion_array >= 0.3) & (max_concentrion_array <= 3000)
valid_ratio =  valid_v & valid_pe & valid_max_c

In [ ]:
# Linear scale and reshape
valid_M = M[valid_ratio]
valid_M = valid_M[0:3000]
valid_A = A[valid_ratio]
valid_A = valid_A[0:3000]
valid_x = x[valid_ratio]
valid_x = valid_x[0:3000]
x_upstream = np.array(valid_x/ 10)
x_upstream = x_upstream.reshape(-1,1)
valid_u = u[valid_ratio]
valid_u = valid_u[0:3000]
valid_D = D[valid_ratio]
valid_D = valid_D[0:3000]
valid_Q = np.array(valid_A*valid_u)

In [ ]:
# Calculate upstream and downstream BTCs using ADE analytical solution
c_upstream = (1e6 * valid_M) / (2 * theta * valid_A * R * np.sqrt(np.pi * valid_D * t_reshape / R)) * np.exp(-((x_upstream - valid_u * t_reshape / R) ** 2) / (4 * valid_D * t_reshape / R))

c_downstream = (1e6 * valid_M) / (2 * theta * valid_A * R * np.sqrt(np.pi * valid_D * t_reshape/ R)) * np.exp(-((valid_x - valid_u * t_reshape / R) ** 2) / (4 * valid_D * t_reshape / R))

In [ ]:
X = torch.tensor(c_upstream,dtype=torch.float32).to(device)
y = torch.tensor(c_downstream,dtype=torch.float32).to(device)

In [ ]:
# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_size)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
subset_sizes = [100, 200, 300, 400, 500, 600, 800, 1000, 1200, 1500, 2000,3000]
final_losses_mean = []
train_losses_mean = []
cc_results_mean = []


r2_results_peak_mean = []
r2_results_tpeak_mean = []
r2_results_m0_mean = []
r2_results_m1_mean = []
mape_results_peak = []
mape_results_tpeak = []
mape_results_m0 = []
mape_results_m1 = []
bias_results_peak = []
bias_results_tpeak = []
bias_results_m0 = []
bias_results_m1 = []


# Set the number of repetitions
repeats = 5

for n_samples in subset_sizes:
    print(f"\nTraining with {n_samples} samples ({repeats} repetitions)...")

    final_losses_rep = []
    train_losses_rep = []
    cc_results_rep = []

    r2_results_peak_rep = []
    r2_results_tpeak_rep = []
    r2_results_m0_rep = []
    r2_results_m1_rep = []
    mape_results_peak_rep = []
    mape_results_tpeak_rep = []
    mape_results_m0_rep = []
    mape_results_m1_rep = []
    bias_results_peak_rep = []
    bias_results_tpeak_rep = []
    bias_results_m0_rep = []
    bias_results_m1_rep = []


    for rep in range(repeats):
        # 1. Take a subset of the data
        subset_X = X[:n_samples, :]
        subset_y = y[:n_samples, :]

        # 2. Create TensorDataset and split
        dataset = TensorDataset(subset_X, subset_y)
        train_size = int(0.7 * n_samples)
        val_size = int(0.2 * n_samples)
        test_size = n_samples - train_size - val_size
        set_seed(rep)  # Change seed each repetition for diversity
        train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=True)

        # 3. Initialize new model each time
        input_size = subset_X.shape[1]
        output_size = subset_y.shape[1]
        model = MLP(input_size, output_size).to(device)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.5)

        # 4. Train model
        epochs = 1000
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0
            for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            scheduler.step()

        avg_train_loss = running_loss / len(train_loader)
        train_losses_rep.append(avg_train_loss)

        # 5. Evaluate on validation set
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        avg_val_loss = val_loss / len(val_loader)
        final_losses_rep.append(avg_val_loss)

        # 6. Evaluate on test set
        model.eval()
        actual_curves = []
        predicted_curves = []
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                actual_curves.extend(batch_y.cpu().numpy())
                predicted_curves.extend(outputs.cpu().numpy())

        actual_curves = np.array(actual_curves)
        predicted_curves = np.array(predicted_curves)
        predicted_curves[predicted_curves < 0] = 0



        # CROSS CORELATION
        max_lag = 100
        use_normalize = 'zscore'
        cc_results = []
        for i in range(actual_curves.shape[0]):
            a = actual_curves[i]
            b = predicted_curves[i]
            res = best_lag_correlation(a, b, max_lag=max_lag, normalize=use_normalize)
            cc_results.append(res)
        cc_mean_corr = np.mean([r['corr'] for r in cc_results])
        cc_results_rep.append(cc_mean_corr)



        # Peak concentration
        actual_peak = np.max(actual_curves, axis=1)
        predicted_peak = np.max(predicted_curves, axis=1)
        r2_peak = r2_score(actual_peak, predicted_peak)
        r2_results_peak_rep.append(r2_peak)
        mape_peak = np.mean(np.abs((predicted_peak - actual_peak) / actual_peak)) * 100
        mape_results_peak_rep.append(mape_peak)
        bias_peak = np.mean(predicted_peak) / np.mean(actual_peak)
        bias_results_peak_rep.append(bias_peak)

        # Time to peak
        actual_peak_time = t_numpy[np.argmax(actual_curves, axis=1)]
        predicted_peak_time = t_numpy[np.argmax(predicted_curves, axis=1)]
        r2_tpeak = r2_score(actual_peak_time, predicted_peak_time)
        r2_results_tpeak_rep.append(r2_tpeak)
        mape_tpeak = np.mean(np.abs((predicted_peak_time - actual_peak_time) / actual_peak_time)) * 100
        mape_results_tpeak_rep.append(mape_tpeak)
        bias_tpeak = np.mean(predicted_peak_time) / np.mean(actual_peak_time)
        bias_results_tpeak_rep.append(bias_tpeak)

        #M0
        actual_m0 = []
        predicted_m0 = []
        for i in range(actual_curves.shape[0]):
            actual_m0.append(np.trapezoid(actual_curves[i,:],t_reshape))
            predicted_m0.append(np.trapezoid(smooth_peak(predicted_curves[i,:]),t_reshape))
        actual_m0 = np.array(actual_m0)
        predicted_m0 = np.array(predicted_m0)
        r2_m0 = r2_score(actual_m0, predicted_m0)
        r2_results_m0_rep.append(r2_m0)
        mape_m0 = np.mean(np.abs((predicted_m0 - actual_m0) / actual_m0)) * 100
        mape_results_m0_rep.append(mape_m0)
        bias_m0 = np.mean(predicted_m0) / np.mean(actual_m0)
        bias_results_m0_rep.append(bias_m0)

        #M1
        actual_m1 = []
        predicted_m1 = []
        for i in range(actual_curves.shape[0]):
            actual_moment = actual_curves[i,:]*t_reshape
            predicted_moment = smooth_peak(predicted_curves[i,:])*t_reshape
            actual_m1.append(np.trapezoid(actual_moment,t_reshape))
            predicted_m1.append(np.trapezoid(predicted_moment,t_reshape))
        actual_m1 = np.array(actual_m1)
        predicted_m1 = np.array(predicted_m1)
        r2_m1 = r2_score(actual_m1, predicted_m1)
        r2_results_m1_rep.append(r2_m1)
        mape_m1 = np.mean(np.abs((predicted_m1 - actual_m1) / actual_m1)) * 100
        mape_results_m1_rep.append(mape_m1)
        bias_m1 = np.mean(predicted_m1) / np.mean(actual_m1)
        bias_results_m1_rep.append(bias_m1)



    # After the repetitions, take the mean
    final_losses_mean.append(np.mean(final_losses_rep)) # val loss
    train_losses_mean.append(np.mean(train_losses_rep)) # trian loss
    cc_results_mean.append(np.mean(cc_results_rep)) #cc
    r2_results_peak_mean.append(np.mean(r2_results_peak_rep)) # peak r2
    r2_results_tpeak_mean.append(np.mean(r2_results_tpeak_rep)) #tpeak r2
    r2_results_m0_mean.append(np.mean(r2_results_m0_rep)) #m0 r2
    r2_results_m1_mean.append(np.mean(r2_results_m1_rep)) #m1 r2
    mape_results_peak.append(np.mean(mape_results_peak_rep)) # mape peak
    mape_results_tpeak.append(np.mean(mape_results_tpeak_rep))  # mape tpeak
    mape_results_m0.append(np.mean(mape_results_m0_rep)) # mape m0
    mape_results_m1.append(np.mean(mape_results_m1_rep))  # mape m1
    bias_results_peak.append(np.mean(bias_results_peak_rep)) # bias peak
    bias_results_tpeak.append(np.mean(bias_results_tpeak_rep)) # bias tpeak
    bias_results_m0.append(np.mean(bias_results_m0_rep)) # bias m0
    bias_results_m1.append(np.mean(bias_results_m1_rep)) # bias m1


    print(f"Finished {n_samples} samples: Avg Validation Loss = {np.mean(final_losses_rep):.6f}, Avg Peak R² = {np.mean(r2_results_peak_rep):.6f}, Avg Tpeak R² = {np.mean(r2_results_tpeak_rep):.6f}, Avg M0 R² = {np.mean(r2_results_m0_rep):.6f}, Avg CC = {np.mean(cc_results_rep):.6f}")
    print(f"peak mape = {np.mean(mape_results_peak_rep)}, peak bias = {np.mean(bias_results_peak_rep)}")
    print(f"tpeak mape = {np.mean(mape_results_tpeak_rep)}, tpeak bias = {np.mean(bias_results_tpeak_rep)})")
    print(f"m0 mape = {np.mean(mape_results_m0_rep)}, m0 bias = {np.mean(bias_results_m0_rep)}")
    print(f"m1 mape = {np.mean(mape_results_m1_rep)}, m1 bias = {np.mean(bias_results_m1_rep)}")


Training with 100 samples (5 repetitions)...
Finished 100 samples: Avg Validation Loss = 423.709606, Avg Peak R² = 0.798980, Avg Tpeak R² = 0.938410, Avg M0 R² = 0.593927, Avg CC = 0.863300
peak mape = 32.08690643310547, peak bias = 0.7062751650810242
tpeak mape = 10.974352936932345, tpeak bias = 1.0012189204481756)
m0 mape = 23.504820955321545, m0 bias = 1.0479491698637091
m1 mape = 24.064758290179263, m1 bias = 1.0014431566137136

Training with 200 samples (5 repetitions)...
Finished 200 samples: Avg Validation Loss = 511.072139, Avg Peak R² = 0.854124, Avg Tpeak R² = 0.947570, Avg M0 R² = 0.967603, Avg CC = 0.877541
peak mape = 20.3961238861084, peak bias = 0.835231602191925
tpeak mape = 8.423939415986817, tpeak bias = 0.9525974168658233)
m0 mape = 15.254164326770285, m0 bias = 0.9649763247453166
m1 mape = 15.983539714380516, m1 bias = 0.9748281823423796

Training with 300 samples (5 repetitions)...
Finished 300 samples: Avg Validation Loss = 111.652875, Avg Peak R² = 0.859135, Avg

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, final_losses_mean, marker='o', label='Validation loss')
plt.plot(subset_sizes, train_losses_mean, marker='x', label='Train loss')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("Loss (Final Epoch)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, cc_results_mean, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("Cross-Correlation Score")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, r2_results_peak_mean, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("R$^2$ for peak concentration")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, mape_results_peak, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("MAPE for peak concentration (%)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, bias_results_peak, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("Bias ratio for peak concentration")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, r2_results_tpeak_mean, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("R$^2$ for time to peak")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, mape_results_tpeak, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("MAPE for time to peak (%)")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, bias_results_tpeak, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("Bias ratio for time to peak")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, r2_results_m0_mean, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("R$^2$ for M0 ")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, mape_results_m0, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("MAPE for M0 (%) ")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, bias_results_m0, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("Bias ratio for M0")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, r2_results_m1_mean, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("R$^2$ for M1 ")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, mape_results_m1, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("MAPE for M1 (%) ")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(subset_sizes, bias_results_m1, marker='o')
plt.xlabel("Number of synthetic breakthrough curves")
plt.ylabel("Bias ratio for M1 ")
plt.grid(True)
plt.show()

In [ ]:
x_values = subset_sizes

val_loss = final_losses_mean
cross_corr = cc_results_mean

mape_results_peak = mape_results_peak
mape_results_tpeak = mape_results_tpeak
mape_results_m0 = mape_results_m0
mape_results_m1 = mape_results_m1

fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=False)

# (a) Validation loss
axes[0].plot(x_values, val_loss, marker='o', color='#1f77b4')
axes[0].set_xlabel("Number of synthetic breakthrough curves")
axes[0].set_ylabel("Validation Loss (Final epoch)")
axes[0].grid(True)
axes[0].set_title("(a)", loc='left', fontsize=13, fontweight='bold')

# (b) Cross-correlation
axes[1].plot(x_values, cross_corr, marker='s', color='#ff7f0e')
axes[1].set_xlabel("Number of synthetic breakthrough curves")
axes[1].set_ylabel("Cross-Correlation Score")
axes[1].grid(True)
axes[1].set_title("(b)", loc='left', fontsize=13, fontweight='bold')

# (c) MAPE metrics
axes[2].plot(x_values, mape_results_peak, marker='o', label="Peak")
axes[2].plot(x_values, mape_results_tpeak, marker='s', label="Time to Peak")
axes[2].plot(x_values, mape_results_m0, marker='^', label=r"$M_0$")
axes[2].plot(x_values, mape_results_m1, marker='d', label=r"$M_1$")
axes[2].set_xlabel("Number of synthetic breakthrough curves")
axes[2].set_ylabel("MAPE (%)")
axes[2].legend()
axes[2].grid(True)
axes[2].set_title("(c)", loc='left', fontsize=13, fontweight='bold')

plt.tight_layout(w_pad=3.5)
plt.show()

In [ ]:

save_path = "/content/drive/MyDrive/"  # Update to your local path
os.makedirs(save_path, exist_ok=True)

df = pd.DataFrame({
    "final_losses_mean": final_losses_mean,
    "train_losses_mean": train_losses_mean,
    "r2_results_mean": r2_results_mean,
    "r2_results_peak_mean": r2_results_peak_mean,
    "r2_results_tpeak_mean": r2_results_tpeak_mean,
    "r2_results_m0_mean": r2_results_m0_mean,
    "r2_results_m1_mean": r2_results_m1_mean,
    "cc_results_mean": cc_results_mean,
    "rmse_results_mean": rmse_results_mean,
    "mape_results_peak": mape_results_peak,
    "mape_results_tpeak": mape_results_tpeak,
    "mape_results_m0": mape_results_m0,
    "mape_results_m1": mape_results_m1,
    "bias_results_peak": bias_results_peak,
    "bias_results_tpeak": bias_results_tpeak,
    "bias_results_m0": bias_results_m0,
})

# 4. Save CSV in the specified path
csv_file = os.path.join(save_path, "results_quantitymlp.csv")
df.to_csv(csv_file, index=False)

